In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
import re
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_absolute_percentage_error
import plotly.graph_objects as go

# **ТИП ТОПЛИВА И ОБЬЕМ ДВИГАТЕЛЯ. ЛИНЕЙНАЯ МОДЕЛЬ**

In [2]:
processed_df = pd.read_csv('processed_df.csv')

In [3]:
processed_df.columns

Index(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW', 'Nissan',
       'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo', 'Kia', 'Force',
       'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'year', 'selling_price',
       'km_driven', 'Diesel', 'Individual', 'Dealer', 'Manual', 'First Owner',
       'Second Owner', 'Third Owner', 'Fourth & Above Owner', 'milage',
       'engine', 'max_power', 'torque', 'RPM_min', 'seats'],
      dtype='object')

**ВЫБОР ДВУХ ПРИЗНАКОВ- ТИП ТОПЛИВА И ОБЬЕМ ДВИГАТЕЛЯ**

In [4]:
processed_df = processed_df.loc[:, ['Diesel', 'engine', 'selling_price']].dropna()

**НОРМАЛИЗУЕМ ПРИЗНАКИ ПО СРЕДСТВАМ SNADART SCALLER**

In [5]:
scaler = StandardScaler()
x = scaler.fit_transform(processed_df.loc[:, ['Diesel', 'engine']])
y = processed_df['selling_price']

**ДЛЯ НАЧАЛА ПОПРОБУЕМ ОБУЧИТЬ НА ПРИМЕРЕ ОДНОЙ СТЕПЕНИ ПОЛИНОМА**

In [6]:
poly = PolynomialFeatures(2)
x_poly = poly.fit_transform(x)
x_poly

array([[ 1.        ,  0.90175469, -0.42334643,  0.81316153, -0.38175463,
         0.1792222 ],
       [ 1.        ,  0.90175469,  0.07346712,  0.81316153,  0.06624932,
         0.00539742],
       [ 1.        , -1.10894904,  0.07147987,  1.22976797, -0.07926753,
         0.00510937],
       ...,
       [ 1.        ,  0.90175469, -0.42334643,  0.81316153, -0.38175463,
         0.1792222 ],
       [ 1.        ,  0.90175469, -0.12923281,  0.81316153, -0.11653629,
         0.01670112],
       [ 1.        ,  0.90175469, -0.12923281,  0.81316153, -0.11653629,
         0.01670112]])

In [7]:
reg_1 = LinearRegression(fit_intercept=False)
cv_results = cross_validate(reg_1, x_poly, y, cv=5, return_train_score=True, scoring='neg_mean_absolute_percentage_error')
print(cv_results['test_score'])
print(cv_results['train_score'])

[-0.91299339 -0.9289455  -0.86285544 -0.96217484 -0.93307167]
[-0.91021517 -0.93930558 -0.89954876 -0.9289453  -0.91526563]


# **TODO:**

1. запилить в цикл по степени полинома, можно реализовать вычисление своей функции потерь (smape, mape, mae?)
!!! кросс валидация sklearn.model_selection.cross_validate!!!
2. пострроить гафики, где х - степень полинома, у - потери (разными метриками)

# **ЗАВИСИМОСТЬ СТЕПЕНИ ПОЛИНОМА ОТ ПОТЕРЬ**

**1. СОЗДАДИМ СПИСОК СТЕПЕНЕЙ ПОЛИНОМА**

In [8]:
poly_power = np.arange(1, 21, 1)
poly_power

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

**СОЗДАДИМ ДФ В КОТОРЫЙ БУДЕМ ЗАПИСЫВАТЬ РЕЗУЛЬТАТЫ**

**ЦИКЛ С ПЕРЕБОРОМ СТЕПЕНЕЙ ПОЛИНОМА. ПОТЕРИ- MAPE**

In [9]:
poly_loss_df = pd.DataFrame({'poly': [], 'loss_train': [], 'loss_test': []})
for p in poly_power:
    # генерация признаков с полиномом степени p
    polynom = PolynomialFeatures(p)
    x_poly = polynom.fit_transform(x)
    # оздание обьекта линейной модели обучения. fit_intercept=False для того, что бы модель не добавляла единичный столбец к матрице признаков для свободного коэффициента w
    reg = LinearRegression(fit_intercept=False)
    # применяем кросс-валидацию для поиска потерь
    cv_results = cross_validate(reg_1, x_poly, y, cv=5, return_train_score=True, scoring='neg_mean_absolute_percentage_error')
    # записываем стредние потери для тестовых и тренировочных датасетов в датафрейм
    poly_loss_df = poly_loss_df.append({'poly': p, 'loss_train': np.mean(np.array(cv_results['train_score'])), 'loss_test': np.mean(np.array(cv_results['test_score']))}, ignore_index=True)
poly_loss_df

,poly,loss_train,loss_test
0,1.0,-0.880270,-0.880490
1,2.0,-0.918656,-0.920008
2,3.0,-0.888607,-0.889161
3,4.0,-0.978393,-0.980370
4,5.0,-0.896167,-0.898502
5,6.0,-0.893544,-0.897627
6,7.0,-0.806662,-0.811735
7,8.0,-0.828495,-0.835215
8,9.0,-0.774560,-0.779498
9,10.0,-0.768946,-0.794867


In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=poly_loss_df['poly'], y=poly_loss_df['loss_train'], mode='lines', name='train_score'))
fig.add_trace(go.Scatter(x=poly_loss_df['poly'], y=poly_loss_df['loss_test'], mode='lines', name='test_score'))
fig.update_layout(width = 1400, height=1000, yaxis_range=[-1, -0.6])
fig.write_html('mape_disel_engine.html')

**ПОТЕРИ MAE**

In [11]:
poly_loss_df_mae = pd.DataFrame({'poly': [], 'loss_train': [], 'loss_test': []})
for p in poly_power:
    # генерация признаков с полиномом степени p
    polynom = PolynomialFeatures(p)
    x_poly = polynom.fit_transform(x)
    # оздание обьекта линейной модели обучения. fit_intercept=False для того, что бы модель не добавляла единичный столбец к матрице признаков для свободного коэффициента w
    reg = LinearRegression(fit_intercept=False)
    # применяем кросс-валидацию для поиска потерь
    cv_results = cross_validate(reg_1, x_poly, y, cv=5, return_train_score=True, scoring='neg_mean_absolute_error')
    # записываем стредние потери для тестовых и тренировочных датасетов в датафрейм
    poly_loss_df_mae = poly_loss_df_mae.append({'poly': p, 'loss_train': np.mean(np.array(cv_results['train_score'])), 'loss_test': np.mean(np.array(cv_results['test_score']))}, ignore_index=True)
poly_loss_df_mae

,poly,loss_train,loss_test
0,1.0,-373648.649016,-3.738408e+05
1,2.0,-376762.851774,-3.771661e+05
2,3.0,-373177.162682,-3.736786e+05
3,4.0,-386192.440754,-3.868821e+05
4,5.0,-358810.034831,-3.604825e+05
5,6.0,-355618.065175,-3.578117e+05
6,7.0,-335616.677432,-3.380178e+05
7,8.0,-338702.654446,-3.430534e+05
8,9.0,-318697.105731,-3.208235e+05
9,10.0,-315965.278104,-3.389272e+05


In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=poly_loss_df_mae['poly'], y=poly_loss_df_mae['loss_train'], mode='lines', name='train_score'))
fig.add_trace(go.Scatter(x=poly_loss_df_mae['poly'], y=poly_loss_df_mae['loss_test'], mode='lines', name='test_score'))
fig.update_layout(width = 1400, height=1000, yaxis_range=[-400000, -250000])
fig.write_html('mae_disel_engine.html')

# **ПРОБЕГ И КРУТЯЩИЙ МОМЕНТ. ЛИНЕЙНАЯ МОДЕЛЬ**

In [13]:
processed_df = pd.read_csv('processed_df.csv')
kmdriven_torque_data = processed_df.loc[:, ['km_driven', 'torque']]